## 课程大纲
1. [协程](#1)
2. [asyncio模块进行异步编程](#2)
3. [实战案例](#3)

### <a id = "1" >1.协程</a>
+ 定义：
    ```text
    协程（Coroutine），也可以被称为微线程，是一种用户态内的上下文切换技术。
    简而言之，其实就是通过一个线程实现代码块互相切换执行。
    ```

+ 协程实现的方法：
    + greenlet
    + yield关键字
    + asyncio装饰器（py3.4之后）
    + async、await关键字（py3.5之后，推荐）

+ 协程的意义：
    ```
    在一个线程中如果遇到IO等待时间，线程不会傻傻等待，利用空闲的时间再去做些其他时间
    ```

#### 1.1 greenlet实现协程

In [ ]:
from greenlet import greenlet


def func1():
    print(1)
    gr2.switch()
    print(3)
    gr2.switch()


def func2():
    print(2)
    gr1.switch()
    print(4)
    gr1.switch()


gr1 = greenlet(func1)
gr2 = greenlet(func2)
gr1.switch()

#### 1.2 yield关键字实现协程

In [ ]:
def func1():
    yield 1
    yield from func2()
    yield 3


def func2():
    yield 2
    yield 4


[print(_) for _ in func1()]

#### 1.3 asyncio实现协程（Python3.4及之后的版本使用）

In [ ]:
import asyncio


@asyncio.coroutine
def func1():
    print(1)
    yield from asyncio.sleep(2)  # 遇到IO耗时操作，自动切换到tasks中的其他任务上
    print(3)


@asyncio.coroutine
def func2():
    print(2)
    yield from asyncio.sleep(2)  # 遇到IO耗时操作，自动切换到tasks中的其他任务上
    print(4)


tasks = [
    asyncio.ensure_future(func1()),
    asyncio.ensure_future(func2())
]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))
# 注意：asyncio遇到IO自动切换


#### 1.4 async & await关键字（Python3.5及之后的版本使用）

In [ ]:
import asyncio


async def func1():
    print(1)
    await asyncio.sleep(2)  # 遇到IO耗时操作，自动切换到tasks中的其他任务上
    print(3)


async def func2():
    print(2)
    await asyncio.sleep(2)  # 遇到IO耗时操作，自动切换到tasks中的其他任务上
    print(4)


tasks = [
    asyncio.ensure_future(func1()),
    asyncio.ensure_future(func2())
]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))


### <a id = "2">2. asyncio模块进行异步编程</a>

#### 2.1 事件循环
+ 理解：就是一个死循环，去检测并执行某些代码

+ 伪代码：
    ```
    任务列表 = [任务1，任务2，...，任务N]

    while True:
        可执行的任务列表，已完成的任务列表 = 去任务列表中检查所有的任务，将“可执行”和“已完成”的任务返回

        for 就绪任务 in 可执行的任务列表:
            执行已就绪的任务

        for 已完成的任务 in 已完成的任务列表:
            在任务列表中移除已完成的任务

        if 任务列表中的任务都已完成:
            终止循环
    ```


#### 2.2 async
+ 协程函数：定义函数的时候格式为： async def 函数名
+ 协程对象：执行 协程函数() 得到的对象。

```python
# 协程函数
async def func():
    pass

# 协程对象
obj = func()

# 注意：执行协程函数时创建的协程对象，函数内部的代码是不会执行的
```


In [1]:
import asyncio


async def func():
    print("快来搞我吧！")


result = func()

# 获取或生成事件循环对象
# loop = asyncio.get_event_loop()
# 执行事件循环对象，直到完成
# loop.run_until_complete(result )

# python3.7之后使用
asyncio.run(result)



RuntimeError: asyncio.run() cannot be called from a running event loop

#### 2.3 await
+ 用法：await + 可等待的对象（协程对象、Future、Task）

+ 注意：await就是等待对象的值得到结果之后再继续向下走。

In [2]:
# 示例1：
import asyncio


async def func():
    print("来玩啊")
    response = await asyncio.sleep(2)
    print("结束", response)


asyncio.run(func())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
# 示例2：
import asyncio


async def others():
    print("start")
    await asyncio.sleep(2)
    print("end")
    return "返回值"


async def func():
    print("执行协程函数内部代码")
    # 遇到IO操作挂起当前协程任务，等IO操作完成之后在继续往下执行。
    # 当前协程挂起时，事件循环可以去执行其他协程任务
    response = await others()
    print("IO请求结束，结果为：", response)


asyncio.run(func())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# 示例3：
import asyncio


async def others():
    print("start")
    await asyncio.sleep(2)
    print("end")
    return "返回值"


async def func():
    print("执行协程函数内部代码")
    # 遇到IO操作挂起当前协程任务，等IO操作完成之后在继续往下执行。
    # 当前协程挂起时，事件循环可以去执行其他协程任务
    response = await others()
    print("IO请求结束，结果为：", response)

    response = await others()
    print("IO请求结束，结果为：", response)

    response = await others()
    print("IO请求结束，结果为：", response)


asyncio.run(func())


#### 2.4 task对象
+ 作用：用来在事件循环中添加多个任务

+ 注意：asyncio.create_task()在Python3.7之后被引入。在Python3.7之前可以使用asyncio.ensure_future()或loop.create_task()

In [ ]:
# 示例1
import asyncio


async def func(i):
    print(i, 'start')
    await asyncio.sleep(2)
    print(i, 'stop')


async def main():
    print("main start")

    # 创建Task对象，将当前执行func函数任务添加到事件循环。
    task1 = asyncio.create_task(func(1))
    task2 = asyncio.create_task(func(2))

    # 当执行某协程遇到IO操作时，会自动化切换执行其他任务
    # 此处的await是等待相对应的协程全部执行完毕并获取结果
    ret1 = await task1
    ret2 = await task2

    print(ret1, ret2)

    print("main stop")


asyncio.run(main())


In [4]:
# 示例2：推荐
import asyncio


async def func(i):
    print(i, 'start')
    await asyncio.sleep(2)
    print(i, 'stop')


async def main():
    print("main start")

    # 创建Task对象，将当前执行func函数任务添加到事件循环。
    tasks = [asyncio.create_task(func(1)),
             asyncio.create_task(func(2)), ]

    # 当执行某协程遇到IO操作时，会自动化切换执行其他任务
    # 此处的await是等待相对应的协程全部执行完毕并获取结果
    done, pending = await asyncio.wait(tasks)
    print(done, pending)

    print("main stop")


asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# 示例3：了解
import asyncio


async def func(i):
    print(i, 'start')
    await asyncio.sleep(2)
    print(i, 'stop')


tasks = [func(1), func(2), ]

asyncio.run(asyncio.wait(tasks))

#### 2.5 Future对象
+ 说明：Task继承Future，Task对象内部await结果的处理基于Future对象来的。


In [ ]:
# 示例1：
import asyncio


async def main():
    # 获取当前事件循环
    loop = asyncio.get_running_loop()

    # 创建一个任务（Future对象），这个任务什么都不干。
    fut = loop.create_future()

    # 等待任务最终结果（Future对象），没有结果则会一直等待下去。
    await fut


asyncio.run(main())


In [ ]:
import asyncio


async def set_after(fut):
    await asyncio.sleep(2)
    fut.set_result("666")


async def main():
    # 获取当前事件循环
    loop = asyncio.get_running_loop()

    # 创建一个任务（Future对象），没有绑定任务行为，则：这个任务永远都不会停止
    fut = loop.create_future()

    # 创建一个任务（Task对象），绑定了set_after函数，函数内部在2s之后，会给fut复制。
    # 手动设置future任务的最终结果，那么fut就可以结束了
    await loop.create_task(set_after(fut))

    # 等待Future对象获取结果
    result = await fut
    print(result)


asyncio.run(main())


#### 2.6 concurrent.futures.Future对象
+ 说明：使用线程池、进程池实现异步操作时用到的对象。

#### 2.7 异步迭代器
```python
import asyncio

class AsyncReader(object):
    """
    自定义异步迭代器，同时也是异步可迭代对象
    """

    def __init__(self):
        self.count = 0

    async def readline(self):
        # 模拟异步操作
        await asyncio.sleep(1)
        self.count +=1
        if self.count == 100:
            return None

        return self.count

    async def __aiter__(self):
        return self

    async def __anext__(self) :
        val = await self.readline()
        if val is None :
            raise StopAsyncIteration

        return val

async def func():
    async for each in AsyncReader():
        print(each)

asyncio.run(func())
```


#### 2.8 异步上下文管理器
```python
import asyncio
class AsyncContentManager(object):
    """定义一个异步上下文管理器对象"""

    def __init__(self):
        self.conn = None

    async def do_something(self):
        # 异步操作数据库
        return "result"

    async def __aenter__(self):
        # 异步连接数据库
        self.conn = await asyncio.sleep(1)

        return self

    async def __aexit__(self,exc_type,exc,tb):
        # 异步关闭数据库
        await asyncio.sleep(1)

async def func():
    async with AsyncContentManager() as f :
        result = await f.do_something()
        print(result)

asyncio.run(func())
```


#### 2.9 uvloop (windows 系统暂不支持)
```python
import asyncio
import uvloop # 需要引入第三方包
# 是asyncio事件循环的代替方案。uvloop事件循环 > 默认asyncio事件循环
# 核心代码
asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())

# 编写正常的asyncio代码即可，内部的事件循环会替换成uvloop的事件循环

asyncio.run(...)
```